<a href="https://colab.research.google.com/github/charlesdtdb/pymc-marketing/blob/main/Catboost_resaerch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from catboost import Pool, CatBoostRegressor
import numpy as np
import pandas as pd

train = pd.read_csv("/kaggle/input/prediction-interval-competition-i-birth-weight/train.csv", index_col="id")
train_X = pd.get_dummies(train, drop_first=True)
train_y = train["DBWT"]
train_X = train_X.drop(["DBWT"], axis=1)

test = pd.read_csv("/kaggle/input/prediction-interval-competition-i-birth-weight/test.csv", index_col="id")
test_X = pd.get_dummies(test, drop_first=True)

train = Pool(train_X, train_y)
test = Pool(test_X)

params = {'learning_rate': 0.01,
          'random_state': 42,
          'colsample_bylevel': 0.5,
          'subsample': 0.5,
          'max_bin': 50,
          'max_depth': 8,
          'loss_function': 'RMSEWithUncertainty',
          'task_type': 'CPU',
          'iterations': 2000,
          'boosting_type': 'Plain',
          'bootstrap_type': 'Bernoulli',
          'verbose': 500,
         }

model = CatBoostRegressor(**params)
model.fit(train)

preds = model.predict(test)
mean_preds, var_preds = preds[:,0], preds[:,1]

sample_submission = pd.read_csv("/kaggle/input/prediction-interval-competition-i-birth-weight/sample_submission.csv")
sample_submission["pi_lower"] = (mean_preds - 1.75*np.sqrt(var_preds)).clip(np.min(train_y), np.max(train_y))
sample_submission["pi_upper"] = (mean_preds + 1.75*np.sqrt(var_preds)).clip(np.min(train_y), np.max(train_y))
sample_submission.to_csv('submission.csv',index=False)
sample_submission.head()